In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../raw_data/df_grouped_rate.csv') #폴더 위치는 상이할 수 있음

In [6]:
y=df.iloc[:,3:4]
X= df.iloc[:,1:].drop(['sales_total'],axis=1)
X.head(3)

,district,code,s_store_no_of_store,s_store_no_of_opening,s_store_no_of_closing,s_work_female,s_float_male,s_float_female,b_facil_total,b_apt_avg_price,b_income_avg_monthly_inc,sales_weekday,sales_female,sales_2030s,sales_06_11,sales_11_14,sales_14_17,sales_17_21,sales_21_24
0,1000001,CS100001,16.5,2,2,1748,211158,145498,129.0,188530154.0,3889111.0,0.707812,0.318123,0.341417,0.007379,0.344443,0.164275,0.345822,0.120878
1,1000001,CS100002,2.0,1,1,1748,211158,145498,129.0,188530154.0,3889111.0,0.734836,0.340414,0.468521,0.006540,0.447948,0.173301,0.363921,0.008290
2,1000001,CS100003,2.0,0,0,1748,211158,145498,129.0,188530154.0,3889111.0,0.914956,0.321958,0.480098,0.000000,0.524393,0.057219,0.227561,0.182710


In [8]:
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,1:])
X_dum = pd.concat([X_dum1, X_dum2],axis=1)


### 회귀분석 (1010개 상권코드)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2) 

# 로버스트 스케일링
rb = RobustScaler()
Xs_train = rb.fit_transform(X_train)
Xs_test = rb.transform(X_test)

# 회귀분석
regressor = LinearRegression()
model = regressor.fit(Xs_train, y_train)

In [11]:
# 정확도 결과값
print(model.score(Xs_train, y_train))
print(model.score(Xs_test, y_test))

print('='*50)
print('='*50)
# 오차 결과값
from sklearn import metrics
y_pred = model.predict(Xs_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.4425188197222003
0.4309835233070318
Mean Absolute Error: 635308846.8475698
Mean Squared Error: 1.535644644916964e+18
Root Mean Squared Error: 1239211299.543772


In [ ]:
#OLS summary를 보기위해 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2, random_state=0) # y 값이 정의되지 않아 df.sales_total 으로 대체하겠습니다.
# 로버스트 스케일링
rb = RobustScaler()
Xs_train = rb.fit_transform(X_train)
Xs_test = rb.transform(X_test)

model = sm.OLS(y_train, Xs_train)
result = model.fit()
print(result.summary())

```
  OLS Regression Results                            
==============================================================================
Dep. Variable:            sales_total   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     66.62
Date:                Sat, 15 Feb 2020   Prob (F-statistic):               0.00
Time:                        16:13:53   Log-Likelihood:            -2.0276e+06
No. Observations:               90612   AIC:                         4.057e+06
Df Residuals:                   89544   BIC:                         4.067e+06
Df Model:                        1067                                         
Covariance Type:            nonrobust                                         
==============================================================================
```

```
==============================================================================
Omnibus:                   116172.839   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         85264506.370
Skew:                           6.589   Prob(JB):                         0.00
Kurtosis:                     152.700   Cond. No.                     1.25e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.28e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
```


### 결론) 1010개의 상권코드와 서비스코드를 원핫인코딩 하면 다중공선성이 발생한다

In [26]:
# Cross Validation 실시(10회)
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict

scores = cross_val_score(model, X_dum, y,scoring='neg_mean_squared_error',n_jobs=-1,cv=10)
print(scores.mean())
scores

-5.1939627337549884e+23


array([-1.49316595e+18, -1.77724911e+18, -1.48891493e+18, -1.72991690e+18,
       -1.61632030e+18, -2.06310464e+18, -1.94553029e+18, -2.28336208e+18,
       -4.54186243e+24, -6.52085910e+23])

In [ ]:
import sklearn
sklearn.metrics.SCORERS.keys()

In [31]:
from sklearn.metrics import r2_score
R2_lr = []
MAE_lr = []
MSE_lr = []
RMSE_lr = []
for i in range(10):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2) 

    # 로버스트 스케일링
    rb = RobustScaler()
    Xs_train = rb.fit_transform(X_train)
    Xs_test = rb.transform(X_test)

    # 회귀분석
    regressor = LinearRegression()
    model = regressor.fit(Xs_train, y_train)

    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    y_pred = model.predict(Xs_test)
    R2_lr.append(r2_score(y_test , y_pred))
    MAE_lr.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_lr.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_lr.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)

1 번째 시도
0.43960408927187367
0.44259013731954644
2 번째 시도
0.4319673796288248
0.467851967080529
3 번째 시도
0.44110374680936393
0.43507485321592126
4 번째 시도
0.4445004482629007
0.42206491872670765
5 번째 시도
0.4470048825554117
0.4100922531433616
6 번째 시도
0.44581264129369047
0.4185232713312772
7 번째 시도
0.4448878215937041
0.4228242516941805
8 번째 시도
0.4384028865182755
0.4484190729706833
9 번째 시도
0.44929374223561613
0.4028637882428904
10 번째 시도
0.44683516373009746
0.41503343183777985


In [32]:
print(R2_lr)
print(MAE_lr)
print(MSE_lr)
print(RMSE_lr)

[0.44259013731954644, 0.467851967080529, 0.4350748532159213, 0.4220649187267077, 0.4100922531433616, 0.4185232713312772, 0.4228242516941805, 0.4484190729706833, 0.4028637882428904, 0.41503343183777985]
[637271723.3349667, 650625361.5460645, 643849356.7065731, 641290271.5972277, 652825710.9467179, 655225875.8360482, 643889460.7855914, 634098102.5302609, 645968725.3911182, 642269354.0035316]
[1.4918839383742797e+18, 1.6796946644326436e+18, 1.6971864079197565e+18, 1.5417979404016678e+18, 1.8189016861481812e+18, 1.7275572564135396e+18, 1.7072931657995963e+18, 1.499558782756685e+18, 1.8193454933958556e+18, 1.7658417092342513e+18]
[1221427009.024395, 1296030348.577009, 1302761070.9258075, 1241691564.1179447, 1348666632.6962276, 1314365723.995243, 1306634289.2330647, 1224564731.9585376, 1348831158.2239845, 1328849769.2494254]


In [35]:
import pickle
from sklearn.externals import joblib
saved_variable = pickle.dumps(R2_lr)
joblib.dump(saved_variable, 'R2_lr.pkl') #피클파일 저장
saved_variable = pickle.dumps(MAE_lr)
joblib.dump(saved_variable, 'MAE_lr.pkl') #피클파일 저장
saved_variable = pickle.dumps(MSE_lr)
joblib.dump(saved_variable, 'MSE_lr.pkl') #피클파일 저장
saved_variable = pickle.dumps(MSE_lr)
joblib.dump(saved_variable, 'RMSE_lr.pkl') #피클파일 저장

['RMSE_lr.pkl']

## 시군구 OLS

In [41]:
df = pd.read_csv("../raw_data/df_grouped_rate.csv", encoding ='utf-8')
cgoongu = pd.read_csv("../raw_data/cgoongoo1.csv", encoding ='utf-8')
df_merged = pd.merge(df, cgoongu, on=['district'])
df_merged.drop(['district','year'], axis=1, inplace =True)

In [42]:
df_merged.columns=[]

Index(['code', 'sales_total', 's_store_no_of_store', 's_store_no_of_opening',
       's_store_no_of_closing', 's_work_female', 's_float_male',
       's_float_female', 'b_facil_total', 'b_apt_avg_price',
       'b_income_avg_monthly_inc', 'sales_weekday', 'sales_female',
       'sales_2030s', 'sales_06_11', 'sales_11_14', 'sales_14_17',
       'sales_17_21', 'sales_21_24', 'cgoongoo'],
      dtype='object')

In [50]:
y=df_merged.iloc[:,1:2]
X= df_merged.drop(['sales_total'],axis=1)
X.head(3)

,code,s_store_no_of_store,s_store_no_of_opening,s_store_no_of_closing,s_work_female,s_float_male,s_float_female,b_facil_total,b_apt_avg_price,b_income_avg_monthly_inc,sales_weekday,sales_female,sales_2030s,sales_06_11,sales_11_14,sales_14_17,sales_17_21,sales_21_24,cgoongoo
0,CS100001,16.5,2,2,1748,211158,145498,129.0,188530154.0,3889111.0,0.707812,0.318123,0.341417,0.007379,0.344443,0.164275,0.345822,0.120878,11110
1,CS100002,2.0,1,1,1748,211158,145498,129.0,188530154.0,3889111.0,0.734836,0.340414,0.468521,0.006540,0.447948,0.173301,0.363921,0.008290,11110
2,CS100003,2.0,0,0,1748,211158,145498,129.0,188530154.0,3889111.0,0.914956,0.321958,0.480098,0.000000,0.524393,0.057219,0.227561,0.182710,11110


In [51]:
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0:-1])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,-1])
X_dum = pd.concat([X_dum1, X_dum2],axis=1)
X_dum.head(3)

,s_store_no_of_store,s_store_no_of_opening,s_store_no_of_closing,s_work_female,s_float_male,s_float_female,b_facil_total,b_apt_avg_price,b_income_avg_monthly_inc,sales_weekday,...,11500,11530,11545,11560,11590,11620,11650,11680,11710,11740
0,16.5,2,2,1748,211158,145498,129.0,188530154.0,3889111.0,0.707812,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1,1,1748,211158,145498,129.0,188530154.0,3889111.0,0.734836,...,0,0,0,0,0,0,0,0,0,0
2,2.0,0,0,1748,211158,145498,129.0,188530154.0,3889111.0,0.914956,...,0,0,0,0,0,0,0,0,0,0


In [53]:
#OLS summary를 보기위해 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2, random_state=0) # y 값이 정의되지 않아 df.sales_total 으로 대체하겠습니다.
# 로버스트 스케일링
rb = RobustScaler()
Xs_train = rb.fit_transform(X_train)
Xs_test = rb.transform(X_test)

model = sm.OLS(y_train, Xs_train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            sales_total   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     725.3
Date:                Thu, 20 Feb 2020   Prob (F-statistic):               0.00
Time:                        19:13:06   Log-Likelihood:            -2.0306e+06
No. Observations:               90612   AIC:                         4.061e+06
Df Residuals:                   90526   BIC:                         4.062e+06
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          5.418e+08   5.01e+06    108.138      0.0

In [54]:
R2_cgoongu = []
MAE_cgoongu = []
MSE_cgoongu = []
RMSE_cgoongu = []
for i in range(10):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2) 

    # 로버스트 스케일링
    rb = RobustScaler()
    Xs_train = rb.fit_transform(X_train)
    Xs_test = rb.transform(X_test)

    # 회귀분석
    regressor = LinearRegression()
    model = regressor.fit(Xs_train, y_train)

    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    y_pred = model.predict(Xs_test)
    R2_cgoongu.append(r2_score(y_test , y_pred))
    MAE_cgoongu.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_cgoongu.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_cgoongu.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)

1 번째 시도
0.4038479294675532
0.41389559737109916
2 번째 시도
0.40885613347879324
0.39482769708405807
3 번째 시도
0.41123339279922366
0.38435428252621195
4 번째 시도
0.408720283143455
0.3935673262695765
5 번째 시도
0.40361897083840564
0.41355839906320047
6 번째 시도
0.40898747136133606
0.39334804165018755
7 번째 시도
0.40830059256332385
0.39453128695885187
8 번째 시도
0.4022047983582724
0.42096328679576445
9 번째 시도
0.4030521308217966
0.41947031819056774
10 번째 시도
0.40996458597327856
0.39134239368624424


In [55]:
import pickle
from sklearn.externals import joblib
saved_variable = pickle.dumps(R2_cgoongu)
joblib.dump(saved_variable, 'R2_cgoongu.pkl') #피클파일 저장
saved_variable = pickle.dumps(MAE_cgoongu)
joblib.dump(saved_variable, 'MAE_cgoongu.pkl') #피클파일 저장
saved_variable = pickle.dumps(MSE_cgoongu)
joblib.dump(saved_variable, 'MSE_cgoongu.pkl') #피클파일 저장
saved_variable = pickle.dumps(RMSE_cgoongu)
joblib.dump(saved_variable, 'RMSE_cgoongu.pkl') #피클파일 저장

['RMSE_cgoongu.pkl']

### Random Forest (상권코드 1010개)

In [58]:
import pandas as pd 
df = pd.read_csv('../raw_data/df_grouped_rate.csv') #폴더 위치는 상이할 수 있음
y=df.iloc[:,3:4]
X= df.iloc[:,1:].drop(['sales_total'],axis=1)
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,1:])
X = pd.concat([X_dum1, X_dum2],axis=1)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
R2_RF = []
MAE_RF = []
MSE_RF = []
RMSE_RF = []
for i in range(10):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    Xs_train = X_train
    Xs_test = X_test
    regr = RandomForestRegressor(n_estimators=50,max_depth=20, n_jobs=-1)
    model=regr.fit(Xs_train, y_train)
    y_pred=model.predict(Xs_test)
    
    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    R2_RF.append(r2_score(y_test , y_pred))
    MAE_RF.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_RF.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_RF.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


1 번째 시도
0.9008015972981063
0.6589950234572413


NameError: name 'metrics' is not defined

In [62]:
import pickle
from sklearn.externals import joblib
saved_variable = pickle.dumps(R2_RF)
joblib.dump(saved_variable, 'R2_RF.pkl') #피클파일 저장
saved_variable = pickle.dumps(MAE_RF)
joblib.dump(saved_variable, 'MAE_RF.pkl') #피클파일 저장
saved_variable = pickle.dumps(MSE_RF)
joblib.dump(saved_variable, 'MSE_RF.pkl') #피클파일 저장
saved_variable = pickle.dumps(RMSE_RF)
joblib.dump(saved_variable, 'RMSE_RF.pkl') #피클파일 저장

['RMSE_RF.pkl']

## Random Forest (시군구)

In [10]:
import pandas as pd

df = pd.read_csv("../raw_data/df_grouped_rate.csv", encoding ='utf-8')
cgoongu = pd.read_csv("../raw_data/cgoongoo1.csv", encoding ='utf-8')
df_merged = pd.merge(df, cgoongu, on=['district'])
df_merged.drop(['district','year'], axis=1, inplace =True)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
y=df_merged.iloc[:,1:2]
X= df_merged.drop(['sales_total'],axis=1)
# X.columns
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0:-1])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,-1])
X = pd.concat([X_dum1, X_dum2],axis=1)
# X_dum.columns

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import r2_score
import numpy as np 

R2_RF_c = []
MAE_RF_c = []
MSE_RF_c = []
RMSE_RF_c = []
for i in range(10):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    Xs_train = X_train
    Xs_test = X_test
    regr = RandomForestRegressor(n_estimators=50,max_depth=20, n_jobs=-1)
    model=regr.fit(Xs_train, y_train)
    y_pred=model.predict(Xs_test)
    
    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    R2_RF_c.append(r2_score(y_test , y_pred))
    MAE_RF_c.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_RF_c.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_RF_c.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


1 번째 시도
0.9014287058748315
0.640904396604018


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


2 번째 시도
0.9004043452937135
0.6647860645829289


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

8 번째 시도
0.9029177972986205
0.6522871360867204


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


9 번째 시도
0.9008034447317089
0.6870329773629011


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


10 번째 시도
0.9021186367139635
0.6709235819285284


In [19]:
import pickle
from sklearn.externals import joblib
saved_variable = pickle.dumps(R2_RF_c)
joblib.dump(saved_variable, 'R2_RF_c.pkl') #피클파일 저장
saved_variable = pickle.dumps(MAE_RF_c)
joblib.dump(saved_variable, 'MAE_RF_c.pkl') #피클파일 저장
saved_variable = pickle.dumps(MSE_RF_c)
joblib.dump(saved_variable, 'MSE_RF_c.pkl') #피클파일 저장
saved_variable = pickle.dumps(RMSE_RF_c)
joblib.dump(saved_variable, 'RMSE_RF_c.pkl') #피클파일 저장

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['RMSE_RF_c.pkl']

### Random Fores dummy 변수?

## LightGBM (1010)

In [21]:
import pandas as pd 
df = pd.read_csv('../raw_data/df_grouped_rate.csv') #폴더 위치는 상이할 수 있음
y=df.iloc[:,3:4]
X= df.iloc[:,1:].drop(['sales_total'],axis=1)
# 더미변수화
X_dum1 = pd.get_dummies(X.iloc[:,0])  #district는 범주형으로 인식 안해서 따로 실시
X_dum2 = pd.get_dummies(X.iloc[:,1:])
X = pd.concat([X_dum1, X_dum2],axis=1)

In [22]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.model_selection import train_test_split
R2_lgb_gbdt = []
MAE_lgb_gbdt = []
MSE_lgb_gbdt = []
RMSE_lgb_gbdt = []
for _ in range(10):
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)
    train_ds = lgb.Dataset(train_x, label = train_y) 
    test_ds = lgb.Dataset(test_x, label = test_y) 

    params = {'learning_rate': 0.01, 
              'max_depth': 16, 
              'boosting': 'gbdt', 
              'objective': 'regression', 
              'metric': 'mse', 
              'is_training_metric': True, 
              'num_leaves': 144, 
              'feature_fraction': 0.9, 
              'bagging_fraction': 0.7, 
              'bagging_freq': 5, 
              'device': 'gpu'
             }
    print(i+1,"번째 시도")
    # 정확도 결과값
    print(model.score(Xs_train, y_train))
    print(model.score(Xs_test, y_test))

    # 오차 결과값
    R2_lgb_gbdt.append(r2_score(y_test , y_pred))
    MAE_lgb_gbdt.append(metrics.mean_absolute_error(y_test, y_pred))
    MSE_lgb_gbdt.append(metrics.mean_squared_error(y_test, y_pred))
    RMSE_lgb_gbdt.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('='*50)
    print('='*50)


10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284
10 번째 시도
0.9021186367139635
0.6709235819285284


In [23]:
import pickle
from sklearn.externals import joblib
saved_variable = pickle.dumps(R2_lgb_gbdt)
joblib.dump(saved_variable, 'R2_lgb_gbdt.pkl') #피클파일 저장
saved_variable = pickle.dumps(MAE_lgb_gbdt)
joblib.dump(saved_variable, 'MAE_lgb_gbdt.pkl') #피클파일 저장
saved_variable = pickle.dumps(MSE_lgb_gbdt)
joblib.dump(saved_variable, 'MSE_lgb_gbdt.pkl') #피클파일 저장
saved_variable = pickle.dumps(RMSE_lgb_gbdt)
joblib.dump(saved_variable, 'RMSE_lgb_gbdt.pkl') #피클파일 저장

['RMSE_lgb_gbdt.pkl']

## combine predictors using stacking
y_pred, y_test

https://scikit-learn.org/stable/auto_examples/ensemble/plot_stack_predictors.html#sphx-glr-auto-examples-ensemble-plot-stack-predictors-py

### boxplot 그리기